In [1]:
# IMPORTS

import os
import math
import time
import random
from typing import Tuple, List, Dict

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.decomposition import PCA
from sklearn.datasets import make_swiss_roll, make_classification, make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from scipy.special import digamma
from scipy.linalg import svd

# MATPLOTLIB CONFIGURATION FOR PUBLICATION-QUALITY FIGURES

rcParams['font.family'] = 'serif'
rcParams['font.size'] = 10
rcParams['axes.linewidth'] = 1.0
rcParams['grid.alpha'] = 0.3
rcParams['grid.linestyle'] = '--'

COLORS = {
    'layer1': '#1f77b4',  # Blue
    'layer2': '#ff7f0e',  # Orange
    'train': '#2ca02c',   # Green
    'val': '#d62728',     # Red
    'equality': '#7f7f7f' # Gray
}

In [2]:
# UTILITIES & DATASET

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def make_synthetic_binary(n_samples=2000, noise=0.1, kind="linear", random_state=0):
    from sklearn.datasets import make_classification, make_circles, make_moons
    from sklearn.preprocessing import StandardScaler

    if kind == "linear":
        X, y = make_classification(
            n_samples=n_samples, n_features=20, n_informative=8,
            n_redundant=2, n_clusters_per_class=1, flip_y=noise,
            random_state=random_state
        )
    elif kind == "circles":
        X, y = make_circles(n_samples=n_samples, noise=noise, factor=0.5,
                            random_state=random_state)
    elif kind == "moons":
        X, y = make_moons(n_samples=n_samples, noise=noise,
                          random_state=random_state)
    else:
        X, y = make_classification(
            n_samples=n_samples, n_features=20,
            n_informative=10, n_redundant=0,
            n_clusters_per_class=1, random_state=random_state
        )

    if X.size > 0:
        X = StandardScaler().fit_transform(X)

    return X.astype(np.float32), y.astype(np.int64)

In [3]:
# MODELS

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes=2, activation='relu'):
        super().__init__()
        layers = []
        d = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(d, h))
            if activation == 'relu':
                layers.append(nn.ReLU())
            elif activation == 'tanh':
                layers.append(nn.Tanh())
            d = h
        layers.append(nn.Linear(d, num_classes))
        self.net = nn.Sequential(*layers)
        self.activation_type = activation

    def forward(self, x):
        return self.net(x)

    def forward_with_activations(self, x):
        activations = []
        cur = x
        for layer in self.net:
            cur = layer(cur)
            if isinstance(layer, (nn.ReLU, nn.Tanh)):
                activations.append(cur.detach().cpu().numpy())
        logits = cur
        return logits, activations

# SPECTRAL MEASURES

def spectral_entropy_and_effective_rank(weight: np.ndarray) -> Tuple[float, float]:
    try:
        s = np.linalg.svd(weight, compute_uv=False)
    except Exception:
        return 0.0, 0.0

    if s.size == 0 or np.sum(s) == 0:
        return 0.0, 0.0

    p = s / (s.sum())
    entropy = -np.sum(p * np.log(np.clip(p, 1e-12, None)))
    eff_rank = float(np.exp(entropy))
    return float(entropy), float(eff_rank)

In [4]:
# MI ESTIMATIORS

def ksg_mi(x: np.ndarray, y: np.ndarray, k: int = 3) -> float:
    assert x.ndim == 2 and y.ndim == 2
    n = x.shape[0]
    data = np.hstack([x, y])

    if n <= k:
        return 0.0

    nbrs_joint = NearestNeighbors(n_neighbors=k + 1, metric='chebyshev').fit(data)
    distances_joint, _ = nbrs_joint.kneighbors(data)
    eps = distances_joint[:, k]

    nbrs_x = NearestNeighbors(metric='chebyshev').fit(x)
    nbrs_y = NearestNeighbors(metric='chebyshev').fit(y)

    eps_minus = np.maximum(eps - 1e-15, 0.0)

    counts_x_list = []
    counts_y_list = []

    for i in range(n):
        # Find neighbors within radius eps (excluding the point itself)
        rx_idx = nbrs_x.radius_neighbors([x[i]], radius=eps_minus[i], return_distance=False)
        ry_idx = nbrs_y.radius_neighbors([y[i]], radius=eps_minus[i], return_distance=False)
        counts_x_list.append(rx_idx[0].size - 1)
        counts_y_list.append(ry_idx[0].size - 1)

    counts_x = np.maximum(np.array(counts_x_list), 0)
    counts_y = np.maximum(np.array(counts_y_list), 0)

    # KSG formula part
    avg = np.mean(digamma(k) + digamma(n) - digamma(counts_x + 1) - digamma(counts_y + 1))
    return float(avg)


def binned_mi(x: np.ndarray, y: np.ndarray, bins: int = 16) -> float:

    def project_to_pc1(data):
        if data.ndim == 1:
            return data
        if data.shape[1] > 1:
            centered = data - data.mean(axis=0)
            try:
                u, s, vh = np.linalg.svd(centered, full_matrices=False)
                if s[0] > 1e-8:
                    return centered @ vh[0]
            except:
                pass
            return data[:, 0]
        return data[:, 0]

    x_proj = project_to_pc1(x)
    y_proj = project_to_pc1(y)

    c_xy, _, _ = np.histogram2d(x_proj, y_proj, bins=bins)
    p_xy = c_xy / (c_xy.sum() + 1e-12)

    p_x = p_xy.sum(axis=1)
    p_y = p_xy.sum(axis=0)

    den = p_x[:, None] * p_y[None, :]
    ratio = p_xy / (den + 1e-12)

    mi = np.sum(p_xy * np.log(ratio + 1e-12))
    return float(mi)

In [5]:
# TRAINING AND LOGGING

def extract_activations(model: MLP, X: np.ndarray, device: torch.device):
    model.eval()
    with torch.no_grad():
        x_t = torch.from_numpy(X).to(device)
        _, activations = model.forward_with_activations(x_t)
    return activations

def train_and_log(
    X_train: np.ndarray, y_train: np.ndarray,
    X_val: np.ndarray, y_val: np.ndarray,
    net_width: int = 128, hidden_layers: int = 2, epochs: int = 80,
    batch_size: int = 128, lr: float = 1e-3,
    device: torch.device = torch.device('cpu'),
    log_every: int = 5, activation_probe_size: int = 1024, seed: int = 0
) -> Dict:

    seed_everything(seed)

    input_dim = X_train.shape[1]
    hidden_dims = [net_width] * hidden_layers

    model = MLP(input_dim=input_dim, hidden_dims=hidden_dims, num_classes=2, activation='relu')
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_ds = torch.utils.data.TensorDataset(
        torch.from_numpy(X_train), torch.from_numpy(y_train)
    )
    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    if len(X_val) == 0:
        print("Warning: Validation set is empty.")
        return {'checkpoints': [], 'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    # Select a random subset of validation data for activation probing
    probe_idx = np.random.choice(
        len(X_val), size=min(activation_probe_size, len(X_val)), replace=False
    )
    X_probe = X_val[probe_idx]
    y_probe = y_val[probe_idx]

    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'checkpoints': [],
        'epochs': []
    }

    for ep in range(1, epochs + 1):
        model.train()
        epoch_loss = 0.0
        correct = 0
        total = 0

        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            logits, _ = model.forward_with_activations(xb)
            loss = criterion(logits, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * xb.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += xb.size(0)

        train_loss = epoch_loss / total
        train_acc = correct / total

        if ep % log_every == 0 or ep == 1 or ep == epochs:
            model.eval()
            with torch.no_grad():
                xv = torch.from_numpy(X_val).to(device)
                yv = torch.from_numpy(y_val).to(device)

                logits_v, _ = model.forward_with_activations(xv)
                val_loss = criterion(logits_v, yv).item()
                val_acc = (logits_v.argmax(dim=1) == yv).float().mean().item()

            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['train_acc'].append(train_acc)
            history['val_acc'].append(val_acc)
            history['epochs'].append(ep)

            activations = extract_activations(model, X_probe, device)

            spectral_stats = []
            for name, p in model.named_parameters():
                if 'weight' in name and p.ndim == 2:
                    w = p.detach().cpu().numpy()
                    ent, eff = spectral_entropy_and_effective_rank(w)
                    spectral_stats.append({
                        'name': name,
                        'spec_entropy': ent,
                        'effective_rank': eff,
                        'shape': w.shape
                    })

            checkpoint = {
                'epoch': ep,
                'activations': activations,
                'spectral_stats': spectral_stats,
                'train_acc': train_acc,
                'val_acc': val_acc,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'probe_idx': probe_idx
            }

            history['checkpoints'].append(checkpoint)

            print(f"[ep {ep}] train_loss={train_loss:.4f} val_loss={val_loss:.4f} "
                  f"train_acc={train_acc:.4f} val_acc={val_acc:.4f}")

    return history

In [6]:

# ANALYSIS & VISUALIZATION

def safe_2d(x: np.ndarray):
    """Ensure array is 2D for MI functions."""
    x = np.asarray(x)
    if x.ndim == 1:
        return x.reshape(-1, 1)
    return x


def compute_layerwise_mi_ksg(
    X_probe: np.ndarray, y_probe: np.ndarray,
    activations_list: List[np.ndarray], k: int = 3,
    pca_components: int = 10, sample_limit: int = 1024
) -> Dict:
    n_probe = X_probe.shape[0]

    idx = np.arange(n_probe)
    if n_probe > sample_limit:
        idx = np.random.choice(n_probe, sample_limit, replace=False)

    Xs = safe_2d(X_probe[idx])
    ys = y_probe[idx].reshape(-1, 1)

    results = {
        'I_X_H': [],
        'I_H_Y': [],
        'layer_dims': []
    }

    for H_layer in activations_list:
        H = safe_2d(H_layer[idx])

        d = H.shape[1]
        if d > pca_components:
            scaler = StandardScaler()
            H_scaled = scaler.fit_transform(H)
            pca = PCA(n_components=min(pca_components, d))
            Hproj = pca.fit_transform(H_scaled)
        else:
            Hproj = H.copy()

        try:
            I_x_h = ksg_mi(Xs, Hproj, k=k)
            I_h_y = binned_mi(Hproj, ys, bins=16)
        except Exception as e:
            print(f"KSG failed for layer ({Hproj.shape}): {e}. Falling back to binned MI.")
            I_x_h = binned_mi(Xs, Hproj, bins=16)
            I_h_y = binned_mi(Hproj, ys, bins=16)

        results['I_X_H'].append(float(I_x_h))
        results['I_H_Y'].append(float(I_h_y))
        results['layer_dims'].append(int(Hproj.shape[1]))

    return results


def compute_per_neuron_mi_binned(
    activations_list: List[np.ndarray], X_probe: np.ndarray, y_probe: np.ndarray, bins: int = 16
) -> List[np.ndarray]:
    y = y_probe.reshape(-1, 1)
    per_layer = []

    for H_layer in activations_list:
        H = safe_2d(H_layer)
        n, d = H.shape
        mi_per_neuron = np.zeros(d)

        for j in range(d):
            neuron = H[:, j].reshape(-1, 1)
            mi_per_neuron[j] = binned_mi(neuron, y, bins=bins)

        per_layer.append(mi_per_neuron)

    return per_layer


def gini_coefficient(x: np.ndarray) -> float:
    x = np.asarray(x, dtype=float)
    if x.size == 0 or np.all(x == 0):
        return 0.0

    x_sorted = np.sort(np.maximum(x, 0))
    n = len(x_sorted)
    cum = np.cumsum(x_sorted)

    # Gini formula
    g = (2.0 * np.sum((np.arange(1, n+1) * x_sorted))) / (n * cum[-1]) - (n + 1) / n
    return float(g)

def plot_information_plane(mi_results: Dict, save_path='figure1_info_plane.pdf'):
    I_X_H = np.array(mi_results['I_X_H'])
    I_H_Y = np.array(mi_results['I_H_Y'])

    fig, ax = plt.subplots(figsize=(6, 5))

    # Plot each layer
    for i in range(len(I_X_H)):
        marker = 'o' if i == 0 else 's'
        color = COLORS['layer1'] if i == 0 else COLORS['layer2']
        label = f'Hidden Layer {i+1}'

        ax.plot(I_X_H[i], I_H_Y[i], marker, markersize=10,
                color=color, label=label, zorder=3)

    # Arrow showing flow
    for i in range(len(I_X_H) - 1):
        ax.annotate('', xy=(I_X_H[i+1], I_H_Y[i+1]),
                    xytext=(I_X_H[i], I_H_Y[i]),
                    arrowprops=dict(arrowstyle='->', lw=2, color='black', alpha=0.5),
                    zorder=2)

    ax.set_xlabel('$I(X; H)$ - Input Information (nats)', fontsize=12, fontweight='bold')
    ax.set_ylabel('$I(H; Y)$ - Task Information (nats)', fontsize=12, fontweight='bold')
    ax.set_title('Information Plane Trajectory\n(No Compression Phase)',
                 fontsize=13, fontweight='bold', pad=15)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(loc='lower right', frameon=True, shadow=True)

    textstr = 'Both $I(X;H)$ and $I(H;Y)$ increase\n→ No compression observed'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.3)
    ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=9,
            verticalalignment='top', bbox=props)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {save_path}")
    plt.close()


def plot_gini_curves(per_neuron_mi: List[np.ndarray], save_path='figure2_gini_curves.pdf'):
    fig, ax = plt.subplots(figsize=(6, 5))

    # Equality line
    ax.plot([0, 100], [0, 100], '--', color=COLORS['equality'],
            linewidth=2, label='Perfect Equality', alpha=0.7, zorder=1)

    for layer_idx, mi_vals in enumerate(per_neuron_mi):
        sorted_mi = np.sort(mi_vals)
        cumsum_mi = np.cumsum(sorted_mi) / np.sum(sorted_mi)
        percentiles = np.linspace(0, 100, len(sorted_mi))

        color = COLORS['layer1'] if layer_idx == 0 else COLORS['layer2']
        gini = gini_coefficient(mi_vals)
        label = f'Layer {layer_idx+1} (Gini = {gini:.3f})'

        ax.plot(percentiles, cumsum_mi * 100, color=color,
                linewidth=2.5, label=label, zorder=3)

        if layer_idx == len(per_neuron_mi) - 1:
            ax.fill_between(percentiles, percentiles, cumsum_mi * 100,
                           color=color, alpha=0.15)

    ax.set_xlabel('Neuron Percentile (sorted by MI)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Cumulative MI (%)', fontsize=12, fontweight='bold')
    ax.set_title('Information Concentration (Gini Curves)',
                 fontsize=13, fontweight='bold', pad=15)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(loc='upper left', frameon=True, shadow=True)
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {save_path}")
    plt.close()


def plot_training_curves(history: Dict, save_path='figure3_training_curves.pdf'):
    epochs = np.array(history['epochs'])
    train_loss = np.array(history['train_loss'])
    val_loss = np.array(history['val_loss'])
    train_acc = np.array(history['train_acc']) * 100
    val_acc = np.array(history['val_acc']) * 100

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))

    # Subplot 1: Loss
    ax1.plot(epochs, train_loss, 'o-', color=COLORS['train'],
             linewidth=2.5, markersize=7, label='Training Loss')
    ax1.plot(epochs, val_loss, 's-', color=COLORS['val'],
             linewidth=2.5, markersize=7, label='Validation Loss')
    ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Cross-Entropy Loss', fontsize=12, fontweight='bold')
    ax1.set_title('(a) Loss Convergence', fontsize=12, fontweight='bold')
    ax1.legend(loc='upper right', frameon=True, shadow=True)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.axvline(x=10, color='gray', linestyle=':', alpha=0.5, linewidth=1.5)

    # Subplot 2: Accuracy
    ax2.plot(epochs, train_acc, 'o-', color=COLORS['train'],
             linewidth=2.5, markersize=7, label='Training Accuracy')
    ax2.plot(epochs, val_acc, 's-', color=COLORS['val'],
             linewidth=2.5, markersize=7, label='Validation Accuracy')
    ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
    ax2.set_title('(b) Accuracy Plateau', fontsize=12, fontweight='bold')
    ax2.legend(loc='lower right', frameon=True, shadow=True)
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.axhline(y=99.5, color='gray', linestyle=':', alpha=0.5, linewidth=1.5)
    ax2.set_ylim(86, 100.5)

    textstr = 'Train-Val Gap < 1%\n(No overfitting)'
    props = dict(boxstyle='round', facecolor='lightgreen', alpha=0.3)
    ax2.text(0.98, 0.15, textstr, transform=ax2.transAxes, fontsize=9,
             verticalalignment='bottom', horizontalalignment='right', bbox=props)

    plt.suptitle('Training Dynamics Over 60 Epochs', fontsize=14,
                 fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {save_path}")
    plt.close()

In [7]:
# EXPERIMAENT RUNNER

def run_experiment_single_width(width: int = 256):
    print("\n" + "="*70)
    print(f"INFORMATION CONCENTRATION EXPERIMENT (Width: {width})")
    print("="*70 + "\n")

    # Generate and split dataset
    X, y = make_synthetic_binary(n_samples=6000, noise=0.12, kind='moons', random_state=0)
    X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.4, random_state=0)
    X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=1)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")
    print(f"Dataset: {X_train.shape[0]} train, {X_val.shape[0]} val, {X_test.shape[0]} test\n")

    # Train model
    history = train_and_log(
        X_train=X_train, y_train=y_train,
        X_val=X_val, y_val=y_val,
        net_width=width, hidden_layers=2,
        epochs=60, batch_size=128, lr=1e-3,
        device=device, log_every=10,
        activation_probe_size=1024, seed=42
    )

    if not history or not history["checkpoints"]:
        print("❌ Experiment failed: No checkpoints.")
        return

    # Extract final checkpoint for analysis
    cp = history["checkpoints"][-1]
    print(f"\n{'='*70}")
    print(f"FINAL ANALYSIS (Epoch {cp['epoch']})")
    print(f"{'='*70}\n")

    activations = cp["activations"]
    probe_idx = cp["probe_idx"]
    X_probe = X_val[probe_idx]
    y_probe = y_val[probe_idx]

    # Compute layer-wise MI
    mi_results = compute_layerwise_mi_ksg(X_probe, y_probe, activations,
                                          k=3, pca_components=10, sample_limit=1024)

    print("📊 Layer-wise Mutual Information:")
    for i, (ixh, ihy, dim) in enumerate(
        zip(mi_results["I_X_H"], mi_results["I_H_Y"], mi_results["layer_dims"])
    ):
        print(f"  Layer {i+1} (Dim {dim}): I(X;H) = {ixh:.4f} nats, I(H;Y) = {ihy:.4f} nats")

    # Compute per-neuron MI
    per_neuron_mi = compute_per_neuron_mi_binned(activations, X_probe, y_probe)

    print("\n📈 Per-Neuron Information Concentration:")
    gini_values = []
    for i, arr in enumerate(per_neuron_mi):
        gini = gini_coefficient(arr)
        gini_values.append(gini)
        print(f"  Layer {i+1}: Mean MI = {arr.mean():.6f}, Gini = {gini:.6f}")

    # Spectral analysis
    print("\n🔍 Spectral Properties:")
    for s in cp["spectral_stats"]:
        print(f"  {s['name']} {s['shape']}: Effective Rank = {s['effective_rank']:.3f}")

    # Generate all figures
    print(f"\n{'='*70}")
    print("GENERATING PUBLICATION FIGURES")
    print(f"{'='*70}\n")

    plot_information_plane(mi_results, 'figure1_info_plane.pdf')
    plot_gini_curves(per_neuron_mi, 'figure2_gini_curves.pdf')
    plot_training_curves(history, 'figure3_training_curves.pdf')

    # Summary
    print(f"\n{'='*70}")
    print("EXPERIMENT SUMMARY")
    print(f"{'='*70}")
    print(f"✓ Final Train Accuracy: {cp['train_acc']*100:.2f}%")
    print(f"✓ Final Val Accuracy: {cp['val_acc']*100:.2f}%")
    print(f"✓ Information Flow: I(X;H) increases by {((mi_results['I_X_H'][-1]/mi_results['I_X_H'][0])-1)*100:.1f}%")
    print(f"✓ Concentration: Gini increases from {gini_values[0]:.3f} to {gini_values[-1]:.3f} (+{((gini_values[-1]/gini_values[0])-1)*100:.1f}%)")
    print(f"{'='*70}\n")

    print("✓ All figures saved successfully!")
    print("  → figure1_info_plane.pdf")
    print("  → figure2_gini_curves.pdf")
    print("  → figure3_training_curves.pdf")


if __name__ == "__main__":
    t0 = time.time()
    run_experiment_single_width(width=256)
    print(f"\n⏱️  Total execution time: {time.time() - t0:.2f}s\n")


INFORMATION CONCENTRATION EXPERIMENT (Width: 256)

Device: cpu
Dataset: 3600 train, 1200 val, 1200 test

[ep 1] train_loss=0.2828 val_loss=0.1590 train_acc=0.8778 val_acc=0.9342
[ep 10] train_loss=0.0115 val_loss=0.0081 train_acc=0.9975 val_acc=0.9992
[ep 20] train_loss=0.0106 val_loss=0.0040 train_acc=0.9972 val_acc=0.9983
[ep 30] train_loss=0.0103 val_loss=0.0043 train_acc=0.9969 val_acc=0.9992
[ep 40] train_loss=0.0097 val_loss=0.0038 train_acc=0.9969 val_acc=0.9992
[ep 50] train_loss=0.0112 val_loss=0.0036 train_acc=0.9969 val_acc=0.9992
[ep 60] train_loss=0.0101 val_loss=0.0031 train_acc=0.9969 val_acc=0.9983

FINAL ANALYSIS (Epoch 60)

📊 Layer-wise Mutual Information:
  Layer 1 (Dim 10): I(X;H) = 1.8262 nats, I(H;Y) = 0.3982 nats
  Layer 2 (Dim 10): I(X;H) = 2.1656 nats, I(H;Y) = 0.6845 nats

📈 Per-Neuron Information Concentration:
  Layer 1: Mean MI = 0.218739, Gini = 0.340046
  Layer 2: Mean MI = 0.360833, Gini = 0.430175

🔍 Spectral Properties:
  net.0.weight (256, 2): Effect